# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment

import os
from azureml.widgets import RunDetails

from azureml.train.automl import AutoMLConfig


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
#ws = Workspace.from_config()
ws = Workspace.get(name="quick-starts-ws-132269")

# choose a name for experiment
experiment_name = 'automl'

experiment=Experiment(ws, experiment_name)

In [ ]:
#  Create compute cluster

cluster_name = "cpu-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
# Set parameters for AutoMLConfig
automl_settings = {
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task='classification',
    compute_target=compute_target,
    debug_log = 'automl_errors.log',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=5,
    **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

In [ ]:
assert(remote_run.get_status() == "Completed")

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)
print(best_run.properties)



In [ ]:
#TODO: Save the best model
best_run.download_file('outputs/model.pkl', 'outputs/model.pkl')
#best_run.download_file('automl_driver.py', 'outputs/automl_driver.py')


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = remote_run.register_model(description='automl', tags= {'area': 'titanic'})
print(remote_run.model_id)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service